In [3]:
import torch
from datasets import Dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

D:\Software\Anaconda\envs\LLM\lib\site-packages\transformers\utils\hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
dataset = Dataset.load_from_disk("../../02_实战演练篇/09_对话机器人/alpaca_data_zh") 
dataset
dataset[0:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

In [3]:
tokenizer = AutoTokenizer.from_pretrained("D:/Study_Date/Modelscope/cache/modelscope/Llama-2-7b-ms")
tokenizer

LlamaTokenizerFast(name_or_path='D:/Study_Date/Modelscope/cache/modelscope/Llama-2-7b-ms', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [4]:
tokenizer.padding_side = "right"
tokenizer.pad_token_id = 2
tokenizer

LlamaTokenizerFast(name_or_path='D:/Study_Date/Modelscope/cache/modelscope/Llama-2-7b-ms', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [5]:
def preprocess_function(examples):
    max_length = 1024
    inputs_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + examples["instruction"], examples["input"]]).strip() + "\n\nAssistant: ",add_special_tokens=False)
    response = tokenizer(examples["output"], add_special_tokens=False)
    inputs_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id] # 模型只需要学习response
    if len(inputs_ids) > max_length:
        inputs_ids = inputs_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    return {"input_ids": inputs_ids, "attention_mask": attention_mask, "labels": labels}

In [6]:
tokenized_datasets = dataset.map(preprocess_function,remove_columns=dataset.column_names)
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:
tokenizer.decode(tokenized_datasets[0]["input_ids"])

'Human: 保持健康的三个提示。\n\nAssistant:  以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>'

In [5]:
model = AutoModelForCausalLM.from_pretrained("D:/Study_Date/Modelscope/cache/modelscope/Llama-2-7b-ms",low_cpu_mem_usage=True,torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OSError: 页面文件太小，无法完成操作。 (os error 1455)

In [4]:
for name, param in model.named_parameters():
    print(name, param.dtype)

NameError: name 'model' is not defined

In [10]:
config = LoraConfig(task_type=TaskType.CAUSAL_LM)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [11]:
model = get_peft_model(model, config)
config
for name, param in model.named_parameters():
    print(name, param.dtype)

base_model.model.model.embed_tokens.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.float16
base_model.model.model.layers.0.self_attn.k_proj.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.float16
base_model.model.model.layers.0.self_attn.o_proj.weight torch.float16
base_model.model.model.layers.0.mlp.gate_proj.weight torch.float16
base_model.model.model.layers.0.mlp.up_proj.weight torch.float16
base_model.model.model.layers.0.mlp.down_proj.weight torch.float16
base_model.model.model.layers.0.input_layernorm.weight torch.float16
base_model.model.model.layers.0.p

In [12]:
model.enable_input_require_grads()# 开启梯度检查点时候必须要开启这个选项
model = model.half()
for name, param in model.named_parameters():
    print(name, param.dtype)

base_model.model.model.embed_tokens.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.float16
base_model.model.model.layers.0.self_attn.k_proj.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.float16
base_model.model.model.layers.0.self_attn.o_proj.weight torch.float16
base_model.model.model.layers.0.mlp.gate_proj.weight torch.float16
base_model.model.model.layers.0.mlp.up_proj.weight torch.float16
base_model.model.model.layers.0.mlp.down_proj.weight torch.float16
base_model.model.model.layers.0.input_layernorm.weight torch.float16
base_model.model.model.layers.0.p

In [13]:
args = TrainingArguments(
    output_dir="./01_半精度训练",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    save_steps=20,
    gradient_checkpointing=True,
    adam_epsilon=1e-4,
)

trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets.select(range(30)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [14]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
D:\Software\Anaconda\envs\LLM\lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


D:\Software\Anaconda\envs\LLM\lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in D:/Study_Date/Modelscope/cache/modelscope/Llama-2-7b-ms - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=3, training_loss=1.39236052831014, metrics={'train_runtime': 204.5355, 'train_samples_per_second': 0.147, 'train_steps_per_second': 0.015, 'total_flos': 176607400919040.0, 'train_loss': 1.39236052831014, 'epoch': 0.8})